In [72]:
%pip install jinja2
%pip install python-dotenv
%pip install requests
%pip install wonderwords

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [73]:
import os
import random
import requests

from dotenv import load_dotenv
from IPython.display import display, Markdown
from jinja2 import Template
from wonderwords import RandomWord

In [74]:
# SPOTIFY API CONSTANTS

load_dotenv()

CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
SPOTIFY_BASE_URL = 'https://api.spotify.com/v1/'

In [75]:
# SPOTIFY API AUTHENTICATION

def get_spotify_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    
    response = requests.post(url, headers=headers, data=data)
    response_data = response.json()
    
    if response.status_code == 200:
        return response_data['access_token']
    else:
        raise Exception(f"Error fetching token: {response_data}")

TOKEN = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
HEADERS = {
        "Authorization": f"Bearer {TOKEN}"
    }
print(TOKEN)

BQAUyw_h1l8YuPy5jti4T4ORUxxzPtdTmPzEKkkT9ZsNGFJn_ti5YUSP0out_bMhaWgF_d-u_zI6izv6_aD6iqRhx7TryDfLL_rYT3dbY_TXSPrKAKE


In [76]:
# SPOTIFY API FUNCTIONS

r = RandomWord()
def get_random_album_id():
    random_word = r.word()
    
    search_url = SPOTIFY_BASE_URL + "search"
    params = {
        "q": random_word,
        "type": "album",
        "tag": "hipster", # to get more obscure results
        "limit": 50
    }
    response = requests.get(search_url, headers=HEADERS, params=params)
    response_data = response.json()
    
    albums = response_data.get('albums', {}).get('items', [])
    non_single_albums = [album for album in albums if album['album_type'] == 'album']
    
    if non_single_albums:
        random_album = random.choice(non_single_albums)
        return random_album['id']
    else:
        return None
    
def get_album_info(album_id):
    url = SPOTIFY_BASE_URL + f"albums/{album_id}"
    response = requests.get(url, headers=HEADERS)
    album = response.json()
    
    album_info = {
        'name': album['name'],
        'id': album['id'],
        'date': album['release_date'],
        'artists': [a['name'] for a in album['artists']],
        'cover': album['images'][0]['url'],
        'genres': album['genres'], # most often not yet classified
        'label': album['label']
    }

    return album_info

def get_album_tracks(album_id):
    url = SPOTIFY_BASE_URL + f"albums/{album_id}/tracks"
    response = requests.get(url, headers=HEADERS)
    response_data = response.json()

    tracks = {}
    for t in response_data.get('items', []):
        track = {
            'name': t['name'],
            'artists': [a['name'] for a in t['artists']]
        }
        tracks[t['track_number']] = track

    return tracks

def get_random_album_information():
    album_id = get_random_album_id()
    album = get_album_info(album_id)
    album['tracks'] = get_album_tracks(album_id)
    return album

def display_album(album):
    if album:       
        display(Markdown(f"**{album['name']}**"))
        display(Markdown(f"![Album Cover]({album['cover']})"))
        display(Markdown(f"**Artists**: {', '.join(album['artists'])}"))
        display(Markdown(f"**Release Date**: {album['date']}"))
        display(Markdown(f"**Genres**: {', '.join(album['genres'])}"))
        display(Markdown(f"**Label**: {album['label']}"))
        display(Markdown(f"**Tracks**"))
        for track_number, track in album['tracks'].items():
            display(Markdown(f"{track_number}. {track['name']} - {', '.join(track['artists'])}"))
    else:
        display(Markdown("No albums found. Try again!"))



In [86]:
# GET RANDOM ALBUM INFORMATION

album = get_random_album_information()
display_album(album)

**Dumpster Fire**

![Album Cover](https://i.scdn.co/image/ab67616d0000b273897f84b6f3f58fe13db69f6a)

**Artists**: Jack Yoder

**Release Date**: 2024-08-13

**Genres**: 

...

6. I Made it to Here - Jack Yoder

7. Ballad of a Fool - Jack Yoder

8. Bottle of Blues - Jack Yoder

9. No Place Else To Go - (Instrumental) - Jack Yoder

10. Southern Dumpster Fire - Jack Yoder

In [87]:
# JINJA TEMPLATE

template = Template("""\
Create a square album cover for the following album:

Album name : {{ album.name }}
Artist{% if album.artists|length > 1 %}s{% endif %} : {{ album.artists | join(', ') }}
Release Date : {{ album.date }}
Label : {{ album.label }}
{% if album.genres %}Genres : {{ album.genres | join(', ') }}{% endif %}
Tracks:
{% for track_number, track in album.tracks.items() %}{{ track_number }}. {{ track.name }} by {{ track.artists | join(', ') }}\n{% endfor %}
""")

print(template.render(album=album))

Create a square album cover for the following album:

Album name : Dumpster Fire
Artist : Jack Yoder
Release Date : 2024-08-13
Label : Southbound Records

Tracks:
1. January by Jack Yoder
2. Bathtub Speed by Jack Yoder
3. Hammer Down by Jack Yoder
4. I'll Still Be Your Friend by Jack Yoder
5. Last Night by Jack Yoder
6. I Made it to Here by Jack Yoder
7. Ballad of a Fool by Jack Yoder
8. Bottle of Blues by Jack Yoder
9. No Place Else To Go - (Instrumental) by Jack Yoder
10. Southern Dumpster Fire by Jack Yoder

